# Projeto Final Exemplo

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

### Alerta

In [2]:
def alerta(nivel, base, etapa, erro=""):
    '''
        Alerta de falha de carregamento de base de dados
    '''
    now = str(datetime.now())

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )


### Banco de Dados

In [3]:
def tabelas_bd():
    '''
        Retorna um dataframe com as tabelas do banco de dados.
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema
def salva_bd(df, nome_tabela):
    '''
        Salva dataframe df na tabela nome_tabela.
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True
def carrega_bd(nome_tabela):
    '''
        Carrega tabela nome_tabela num dataframe. 
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df

### 1 Extração

In [4]:
def get_json_api(url):
    '''
        Request GET url e retorna json de saida
    '''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    
    # Erro
    raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")
    


##### 1.1 Base pokemons_url

In [5]:
def get_base_pokemons_url():
    '''
        request completo iterativo da tabela pokemons_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/pokemon"
    df_pokemons_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_pokemons = pd.DataFrame(json_data['results'])

        #append/concat  
        df_pokemons_full = pd.concat([df_pokemons_full,df_pokemons],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_pokemons_full

##### 1.2 Base habilidades_url

In [6]:
def get_base_habilidades_url():
    '''
        request completo iterativo da tabela habilidades_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/ability"
    df_habilidades_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_habilidade = pd.DataFrame(json_data['results'])

        #append/concat  
        df_habilidades_full = pd.concat([df_habilidades_full,df_habilidade],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_habilidades_full

##### 1.3 Base pokemons

In [7]:
def get_base_pokemons():

    df_pokemons_url = carrega_bd("pokemons_url")

    df_pokemons_full = pd.DataFrame()

    for url in tqdm(df_pokemons_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id              = json_data['id']
        name            = json_data['name']
        abilities       = json_data['abilities']
        height          = json_data['height']
        weight          = json_data['weight']
        base_experience = json_data['base_experience']

        sr_pokemon = pd.Series({
            "url": url,
            "pokemon_id": id,
            "name": name,
            "height": height,
            "weight": weight,
            "base_experience": base_experience,
            "habilidades": str(abilities)
        })

        #append/concat 
        df_pokemons_full = pd.concat([df_pokemons_full,sr_pokemon.to_frame().T],ignore_index=True)
        
    return df_pokemons_full
    

##### 1.4 Base habilidades

In [8]:
def get_base_habilidades():

    df_habilidades_url = carrega_bd("habilidades_url")

    df_habilidades_full = pd.DataFrame()

    for url in tqdm(df_habilidades_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        generation     = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect         = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_habilidade = pd.Series({
            "url": url,
            "ability_id":id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        #append/concat 
        df_habilidades_full = pd.concat([df_habilidades_full,sr_habilidade.to_frame().T],ignore_index=True)

    return df_habilidades_full
    

##### 1.5 Etapa de extracao completa

In [9]:
def etapa_extracao():
    
    #################################################
    print("  > Base pokemons_url:\n")
    try:
        df_pokemons_url = get_base_pokemons_url()
        salva_bd(df_pokemons_url,"pokemons_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "pokemons_url", 
               etapa = "EXTRACAO", 
               erro=e)
    
    #################################################
    print("  > Base habilidades_url:\n")
    try:
        df_habilidades_url = get_base_habilidades_url()
        salva_bd(df_habilidades_url,"habilidades_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "habilidades_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base pokemons:\n")
    try:
        df_pokemons_full = get_base_pokemons()
        salva_bd(df_pokemons_full,"pokemons")
    except Exception as e:
        alerta(nivel = 3, 
               base = "pokemons", 
               etapa = "EXTRACAO", 
               erro=e)

    
    #################################################
    print("  > Base habilidades:\n")
    try:
        df_habilidades_full = get_base_habilidades()
        salva_bd(df_habilidades_full,"habilidades")
    except Exception as e:
        alerta(nivel = 3, 
               base = "habilidades", 
               etapa = "EXTRACAO", 
               erro=e)
        

### 2 Transformação

In [10]:
def etapa_transformacao():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("pokemons")
        #abrindo as linhas por habilidade
        df_pokemons['habilidades'] = df_pokemons['habilidades'].apply(eval)
        df_pokemons_open = df_pokemons.explode('habilidades')
        #buscando id das habilidades
        df_pokemons_open['ability_url'] = df_pokemons_open['habilidades'].apply(lambda row: row['ability']['url'])
        df_pokemons_open['ability_id']  = df_pokemons_open['ability_url'].apply(lambda row: int(row.split('/')[-2]))
        df_pokemons_open = df_pokemons_open.rename(columns={'name':'pokemon_name'})
        df_pokemons_open = df_pokemons_open[['pokemon_id','ability_id','pokemon_name','height','weight','base_experience']]

        #base de habilidades
        df_habilidades = carrega_bd("habilidades")
        df_habilidades = df_habilidades.rename(columns={'name':'ability_name','effect':'ability_effect'})
        df_habilidades = df_habilidades[['ability_id','ability_name','ability_effect']]

        #join da base pokemons e habilidades
        df_pokemons_habilidades = df_pokemons_open.merge(df_habilidades,on=['ability_id'],how='left')
 
        #salva resultado final
        salva_bd(df_pokemons_habilidades,"pokemons_habilidades")
        print("pokemons_habilidades salva")
    except Exception as e:
        alerta(nivel = 2, 
               base = "pokemons_habilidades", 
               etapa = "TRANSFORMACAO", 
               erro=e)
        

### 3 Run

In [11]:
etapa_extracao()

  > Base pokemons_url:

  > Base habilidades_url:/pokemon?offset=1280&limit=10

  > Base pokemons:/api/v2/ability?offset=340&limit=18



100%|██████████| 1281/1281 [04:22<00:00,  4.88it/s]


  > Base habilidades:



100%|██████████| 358/358 [01:37<00:00,  3.65it/s]


In [12]:
etapa_transformacao()

pokemons_habilidades salva


## Load

In [13]:
df = carrega_bd('pokemons_habilidades')
df.head()

,pokemon_id,ability_id,pokemon_name,height,weight,base_experience,ability_name,ability_effect
0,1,65,bulbasaur,7,69,64.0,overgrow,When this Pokémon has 1/3 or less of its HP re...
1,1,34,bulbasaur,7,69,64.0,chlorophyll,Während strong sunlight ist die speed eines Po...
2,2,65,ivysaur,10,130,142.0,overgrow,When this Pokémon has 1/3 or less of its HP re...
3,2,34,ivysaur,10,130,142.0,chlorophyll,Während strong sunlight ist die speed eines Po...
4,3,65,venusaur,20,1000,263.0,overgrow,When this Pokémon has 1/3 or less of its HP re...


In [14]:
df.shape


(2908, 8)